In [26]:
#Libraries
from dotenv import load_dotenv
import os

In [27]:
#Loading Environment Vars
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "ConversationalAIBot"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["HUGGINGFACE_API_KEY"] = os.getenv("HUGGINGFACE_API_KEY")

In [28]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate

model = ChatGroq(
    temperature = 0.2,
    model_name = "deepseek-r1-distill-llama-70b",
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a Research assistant from arxiv"),
        ("human", "{question}")
    ]
)

question = "What does attention is all you need talk about?"

chain = prompt | model

In [29]:
print(chain.invoke({"question": question}).content)

<think>
Okay, so I'm trying to understand what the "Attention is All You Need" paper is about. I've heard it's a big deal in AI and machine learning, especially in NLP, but I'm not exactly sure why. Let me break it down.

First, I know that before this paper, sequence-to-sequence models with encoder-decoders were the norm for tasks like translation. These models used RNNs or LSTMs to handle the sequential nature of the data. But I've also heard that RNNs have some limitations, like handling long-term dependencies and being computationally expensive because they process data sequentially.

So, the paper must be proposing an alternative to RNNs. I remember something about attention mechanisms. Attention allows models to focus on different parts of the input when generating each part of the output. But in this paper, they're saying attention is all you need, which suggests they're getting rid of the RNNs entirely and relying solely on attention.

Wait, how does that work? Without RNNs, ho

#### Loading the CSV file

In [30]:
import pandas as pd
from langchain_core.documents import Document

df = pd.read_csv("bitext-telco-llm-chatbot-training-dataset.csv")
df.head()

,instruction,intent,category,tags,response
0,there are charges on my phone bill that i do n...,dispute_invoice,BILLING,BCELQZ,If you have noticed discrepancies in your bill...
1,"my internet bill is incorrect, can you help me...",dispute_invoice,BILLING,BCILZ,If there is an issue with your bill and you wo...
2,my invoice is not corect challenge it,dispute_invoice,BILLING,BCELQZ,If you find any discrepancies in your invoice ...
3,"I don't recognize some charges on my bill, can...",dispute_invoice,BILLING,BCIL,If you have found charges on your bill that yo...
4,i have to dispute a fucking bill i need help,dispute_invoice,BILLING,BCQW,If you believe there is an error on your bill ...


In [31]:
#Itering rows for organizing: Tried with classical CSV loader, this was efficient
conversation_documents = []

for index, row in df.iterrows():
    content = f""""Instruction: 
    {row['instruction']}
    Intent: {row['intent']}
    Category: {row['category']}
    Tags: {row['tags']}
    Response: {row['response']}"""

    conversation_documents.append(Document(page_content=content, metadata={"intent": row['intent']}))

In [32]:
conversation_documents[:5]

[Document(metadata={'intent': 'dispute_invoice'}, page_content='"Instruction: \n    there are charges on my phone bill that i do not recogniae challenge it\n    Intent: dispute_invoice\n    Category: BILLING\n    Tags: BCELQZ\n    Response: If you have noticed discrepancies in your bill and wish to contest them, please adhere to the following steps:\r\n\r\n1. Log in to your account on {{WEBSITE_URL}}.\r\n2. Navigate to the {{INVOICE_SECTION}} section.\r\n3. Select the bill you wish to dispute.\r\n4. Click on the {{DISPUTE_INVOICE_OPTION}} to dispute the charge.\r\n5. Fill in the required information and submit your dispute.\r\n\r\nOur team will review your submission and get back to you within {{DAYS_NUMBER}} business days.'),
 Document(metadata={'intent': 'dispute_invoice'}, page_content='"Instruction: \n    my internet bill is incorrect, can you help me dispute it ?\n    Intent: dispute_invoice\n    Category: BILLING\n    Tags: BCILZ\n    Response: If there is an issue with your bill

In [33]:
print(conversation_documents[0])

page_content='"Instruction: 
    there are charges on my phone bill that i do not recogniae challenge it
    Intent: dispute_invoice
    Category: BILLING
    Tags: BCELQZ
    Response: If you have noticed discrepancies in your bill and wish to contest them, please adhere to the following steps:

1. Log in to your account on {{WEBSITE_URL}}.
2. Navigate to the {{INVOICE_SECTION}} section.
3. Select the bill you wish to dispute.
4. Click on the {{DISPUTE_INVOICE_OPTION}} to dispute the charge.
5. Fill in the required information and submit your dispute.

Our team will review your submission and get back to you within {{DAYS_NUMBER}} business days.' metadata={'intent': 'dispute_invoice'}


In [34]:
#Vector Embeddings

from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

embedding_model

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [37]:
sample_query = "What is India famous for?"

sample_embedding = embedding_model.embed_query(sample_query)

len(sample_embedding)

384

##### <b> No need to run again <b>

In [13]:
document_embeddings = embedding_model.embed_documents([doc.page_content for doc in conversation_documents])

In [ ]:
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
import faiss

doc_index = faiss.IndexFlatL2(len(sample_embedding))
doc_index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x000001653C64FE70> >

In [19]:
#loading embeddings to vector Store

vector_store = FAISS(
    embedding_function = embedding_model,
    index = doc_index,
    docstore = InMemoryDocstore(),
    index_to_docstore_id = {}
)

In [22]:
vector_store.add_documents(conversation_documents)

['edec9e14-27c9-4a84-9435-ab1e90e1fcb3',
 '47bb6a70-f5bf-4992-8c54-3ffbe8f891e6',
 'd5fb80a4-8b92-48e7-8b7e-41103ef11856',
 '141c4eb5-e2ba-4621-98c1-da41bec106a7',
 '08e42202-a93a-49d9-9f36-c611ca185567',
 'b8a517f4-ded1-44cf-8fd5-6d1c58bc1db1',
 '90bba86a-54ad-4536-bb35-c4e3c6576345',
 'f46d5f8f-783d-47e8-b471-3dabfdefac9f',
 '67c29a0d-16b6-4dca-848f-bb0b072952d4',
 'fc8fa729-0217-407c-bebf-060fcdc6847c',
 'c2f9da08-d375-4e1a-af10-d41e27efd7ea',
 'e6c532b3-353d-4210-8403-fc388c934f86',
 '47851841-d627-41d6-8bd5-59e5c7a9ecef',
 'c516b6df-6bca-43fe-a95b-148c9ed65283',
 'a30c565a-b782-4488-990b-6c0b274acd70',
 '999a4c6b-7fdc-422b-beed-3e5069e8fb57',
 '09cef42c-59f7-4647-bea4-f9d5d6fc5018',
 '652512ad-a20b-4d30-bf95-8731269a40c2',
 '79ea5836-020c-4e80-8381-8f9817aa7618',
 '2df517f5-a91e-43e4-a815-f11e45ffabab',
 '3859ae5e-767b-4b82-8a80-dcfd208f59ea',
 'd2c93839-6b3c-4c66-8116-8e5e6901a5df',
 '39379420-1f6a-47c5-91c9-f3cff0e30b3b',
 '3bf0f22f-049d-4be9-bb4d-a2f5f431d558',
 '87fd62e7-4589-

##### <b>Resume<b>

In [38]:
#saving DB in local

vector_store.save_local("conversation_faiss_index")

In [42]:
#Loading vector store

local_vector_db = FAISS.load_local("C:/Users/bharath.sr.lv/Desktop/ConversationalAI/Conversational_Ai/Data1/conversation_faiss_index",
                                    embeddings=embedding_model,
                                    allow_dangerous_deserialization=True)

Buiding Vanilla RAG

In [ ]:
retriever = local_vector_db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5} 
)

retriever.invoke("I am facing internet issues and my screen is buffereing")

#print(output[0].page_content)

"Instruction: 
    i got to report poor internet signal covedrage i need help
    Intent: report_poor_signal_coverage
    Category: COMPLAINTS
    Tags: BCLQZ
    Response: We recognize the importance of reliable signal coverage for effective communication. Please follow the steps below to report any issues you are experiencing with poor signal coverage:

1. Visit our website at {{WEBSITE_URL}}.
2. Navigate to the {{SUPPORT_SECTION}} section.
3. Select the option labeled {{REPORT_POOR_SIGNAL_COVERAGE_OPTION}}.
4. Fill in the required details, including your location and issue description.
5. Submit the form to notify our team.

Our team will assess your submission and take appropriate measures to enhance network coverage in your vicinity.


In [43]:
from langchain_groq import ChatGroq

groq_model = ChatGroq(
    temperature = 0.2,
    model_name = "llama-3.3-70b-versatile",
)

In [66]:
from langchain_core.prompts import PromptTemplate

rag_prompt_template = """
You are a helpful and professional customer service agent for a telecommunications company.

Your job is to assist customers by answering their questions politely and clearly, based only on the provided context below.

If the context does not contain enough information to answer the question, politely inform the customer that you are escalating their query to a human support agent.

---
Context:
{context}

---
Customer Question:
{question}

---
Response:
"""

prompt = PromptTemplate(
     input_variables = ["context", "question"],
     template = rag_prompt_template   
)

In [67]:
prompt

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\nYou are a helpful and professional customer service agent for a telecommunications company.\n\nYour job is to assist customers by answering their questions politely and clearly, based only on the provided context below.\n\nIf the context does not contain enough information to answer the question, politely inform the customer that you are escalating their query to a human support agent.\n\n---\nContext:\n{context}\n\n---\nCustomer Question:\n{question}\n\n---\nResponse:\n')

##### Format docs function basically formats the retirver output for LLM context

In [68]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [69]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

RAG_chain = (
    {"context": retriever | format_docs,
     "question": RunnablePassthrough()}
     | prompt
     | groq_model
     | StrOutputParser()
)

Output Testing

In [71]:
output = RAG_chain.invoke( "I am facing issues with my 5G connection. My internet speed is slow and I am damn frustrated as I am getting 2G speed.")

print(output)


I apologize for the frustration you're experiencing with your 5G connection. I understand that slow internet speeds can be really frustrating. To help you resolve this issue, I would recommend checking the signal coverage in your area. 

To do this, please follow these steps:

1. Visit our website at {{WEBSITE_URL}} and log in to your account.
2. Navigate to the {{SIGNAL_COVERAGE_SECTION}} section.
3. Enter your address or ZIP code into the search bar.
4. Click the {{SEARCH_BUTTON}} button.

This will provide you with information about the internet services available in your area. If you're still experiencing issues after checking the signal coverage, I would be happy to escalate your query to a human support agent who can further assist you in resolving the problem with your 5G connection. Would you like me to do that?


RAG Testing